# Seasonal Analysis

In [5]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib.pyplot import figure

import datetime as dt
import calmap
import sheworewhat as sww

alt.data_transformers.disable_max_rows()
alt.renderers.enable('mimetype')
# alt.renderers.enable('notebook')

RendererRegistry.enable('mimetype')

In [13]:
def closet_df(path="data/ClosetData.csv"):
    """
    Function to import CSV data and return df with unique identifiers.

    Parameters:
    -----------
        path : str
            Path to CSV file containing closet information.

    Returns:
    --------
        closet : pandas.DataFrame
            Dataframe containing 12 columns: ID, Item, Category, Subcategory,
            Color, Pattern, Brand, Bought, Price, 2023, Cost, Name
    """
    # avoid setting with copy warning
    pd.options.mode.chained_assignment = None

    closet = pd.read_csv(path)

    # create IDs per item
    closet = closet.reset_index().rename(columns={"index": "ID"})

    # format strings to create item name
    closet["Item"] = closet["Item"].map(str.title)
    closet["Brand"] = closet["Brand"].map(str.title)
    closet["PrimaryC"] = closet["Color"].str[:5]

    # create item name
    closet["Name"] = (
        closet["ID"].apply(str)
        + " "
        + closet["Brand"]
        + " "
        + closet["Item"]
        + " - "
        + closet["PrimaryC"]
    )

    # NaNs in 2023 addition column
    for i, value in enumerate(closet["2023"]):
        if value != "Yes":
            closet["2023"].iloc[i] = "No"

    return closet

def worn(closet):
    """
    Function to merge raw closet data and collected 2023 data.

    Parameters
    ----------
        closet : pandas.DataFrame
            Dataframe containing complete closet log.

    Returns
    -------
        worn_df : pandas.DataFrame
            Complete and standardized dataframe containing "ID", "Name", "count", "Item",
            "Category", "Sub-Category", "Color", "Pattern", "Brand", "Cost", "2023"
    """

    sheet_url = "https://docs.google.com/spreadsheets/d/1TP7HQZxiP6as_HHexcwkmDTeXOQQOLbUesZjHwKA-Q4/edit?resourcekey#gid=1344494584"
    url_1 = sheet_url.replace("/edit?resourcekey#gid=", "/export?format=csv&gid=")
    form = pd.read_csv(url_1).drop("Timestamp", axis=1).melt("Date").dropna()

    # extract ID number from value
    form["ID"] = form.value.str.extract("(\d+)").astype(int)

    form_counts = (
        form.groupby(["value", "ID"])
        .count()
        .reset_index()
        .rename(columns={"Date": "count"})
        .drop(["variable"], axis=1)
    )

    # left join closet + df
    worn_df = pd.merge(closet, form_counts, how="left", on="ID")
    worn_df["Name"] = worn_df["Brand"] + " " + worn_df["Item"]
    worn_df = worn_df[
        [
            "ID",
            "Name",
            "count",
            "Item",
            "Category",
            "Sub-Category",
            "Color",
            "Pattern",
            "Brand",
            "Bought",
            "Cost",
            "2023",
            "Price",
        ]
    ]
    worn_df = worn_df.fillna(0).rename(columns={"count": "Count"})
    worn_df["Count"] = worn_df["Count"].astype(int)

    return worn_df

## Determine Four Seasons
- idea: see where pandas datetime cuts quarters

In [14]:
pd.Timestamp(dt.date(2016, 2, 29)).quarter

1

In [60]:
sheet_url = "https://docs.google.com/spreadsheets/d/1TP7HQZxiP6as_HHexcwkmDTeXOQQOLbUesZjHwKA-Q4/edit?resourcekey#gid=1344494584"
url_1 = sheet_url.replace("/edit?resourcekey#gid=", "/export?format=csv&gid=")
form = pd.read_csv(url_1).drop("Timestamp", axis=1).melt("Date").dropna()

# extract ID number from value
form["ID"] = form.value.str.extract("(\d+)").astype(int)
form['Date']= pd.to_datetime(form['Date']) 
form["Day"] = form["Date"].dt.dayofyear


In [61]:
form.sample(3)

,Date,variable,value,ID,Day
196,2023-01-13,Top,2 Plisse Tank - Black,2,13
218,2023-01-09,Top2,28 American Apparel Tank - Black,28,9
244,2023-01-24,Top2,17 Aerie Sports Bra - Black,17,24


In [66]:
# def season(df):
for day in form["Day"]:
    
    # March 20 (79th day of year) = Spring Equinox
    if day in range(79, 172): 
        form["Season"] = "Spring"
    # June 21 (172nd day of year) = Summer Solstice
    elif day in range(172, 265):
        form["Season"] = "Summer"
    # September 22 (265 day of year) = Fall Equinox
    elif day in range(265, 355):
        form["Season"] = "Fall"
    # December 21 (355th day of year) = Winter Solstice
    else:
        form["Season"] = "Winter"
    # also need to include Jan 1 - March 19, 2023
    
form
# return

,Date,variable,value,ID,Day,Season
0,2023-01-01,Accessory,91 Star Earrings,91,1,Winter
3,2023-01-06,Accessory,81 No Brand Gold Hoops - Gold,81,6,Winter
4,2023-01-06,Accessory,81 No Brand Gold Hoops - Gold,81,6,Winter
8,2023-01-09,Accessory,81 No Brand Gold Hoops - Gold,81,9,Winter
10,2023-01-10,Accessory,81 No Brand Gold Hoops - Gold,81,10,Winter
...,...,...,...,...,...,...
333,2023-01-16,Accessory4,94 Ray-Ban Sunglasses,94,16,Winter
344,2023-01-20,Accessory4,94 Ray-Ban Sunglasses,94,20,Winter
345,2023-01-21,Accessory4,94 Ray-Ban Sunglasses,94,21,Winter
348,2023-01-23,Accessory4,92 Taylor Swift Baseball Cap,92,23,Winter


In [ ]:

form['Szn'] = form.Date.map(season)